<a href="https://colab.research.google.com/github/Robdres/Angular/blob/main/Gurufocus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install alive_progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210079 sha256=b5e1de854361568bd514c2adcc4bd4adf512e25964182a6b1dc7465817fc8b6b
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [ ]:
!pip3 install alive_progress
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from alive_progress import alive_bar
import numpy as np

api_token = "<api-token>"

In [ ]:
symbols = ['AAPL','ANET','ASML',"CHGCY","KO","COST","DLICY","DTEGN","LLY","XOM","RACE","GETIB","HON","ITX","JPM","LGFRY","OREP","LULU","MQG","MSFT","MSCI","NESN","NKE","NTDOY","NOVO","ORSTED","RIO","SSDIF","SHIN","TCS","TE","V","WEGE3","WLSN"]

In [ ]:
len(symbols)

36

# Get indicators

In [ ]:
# DO not run again
indicators =requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/indicators')
content = indicators.content
data = json.loads(content.decode('utf8'))

In [ ]:
indicators = pd.read_csv("indicators.csv")
indicators

,Unnamed: 0,key,name
0,0,price,Price
1,1,revenue,Revenue
2,2,revenue_estimate,Revenue with Estimate
3,3,net_income,Net Income
4,4,pettm,PE Ratio (TTM)
...,...,...,...
231,231,guru_buy,Number of Premium Guru Buys
232,232,guru_hold,Number of Premium Guru Holds
233,233,guru_buy_volume,Shares of Premium Guru Buys
234,234,guru_sell_volume,Shares of Premium Guru Sells


# Price

In [ ]:
symbol = "COST"
response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/price?type=a')
content = response.content
price_data = json.loads(content.decode('utf8'))
price = pd.DataFrame(price_data)

In [ ]:
price.columns = ["date",'price']
price["date"] = pd.to_datetime(price["date"])

# Indicators

In [ ]:
indicators = pd.read_csv("indicators.csv")
indicators
indicator = indicators.iloc[4:20].key
indicators_list = list(indicators['key'])
indicator = indicators_list[0:10]
indicator

['price',
 'revenue',
 'revenue_estimate',
 'net_income',
 'pettm',
 'price_to_owner_earnings',
 'ps_ratio',
 'pb_ratio',
 'peg_ratio',
 'yield']

In [ ]:
indicator = indicators_list[0:10]
aux = ["enterprise_value_to_revenue","enterprise_value_to_ebitda","enterprise_value_to_ebit","earning_yield_greenblatt","rate_of_return_value","shiller_pe_ratio","price_to_tangible_book"
       ,"price_to_free_cash_flow","price_to_operating_cash_flow"]
for i in aux:
    indicator.append(i)
indicator.pop(0)
indicator.pop(0)
indicator.pop(0)
a =indicator.pop(0)

In [ ]:
data=price
with alive_bar(len(indicator),force_tty=True) as bar:
    for i in range(len(indicator)):
        bar()
        ind = indicator[i]
        response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/{ind}?type=a')
        content = response.content
        indicator_data = json.loads(content.decode('utf8'))
        if(len(indicator_data)==0):
            continue
        else:
            indicator_df = pd.DataFrame(indicator_data)
            indicator_df[0] = pd.to_datetime(indicator_df[0])
            indicator_df.columns = ["date",ind]
            data = pd.merge(data,indicator_df,how="left",on=["date"])
data.to_csv(symbol+".csv")

|████████████████████████████████████████| 15/15 [100%] in 23.8s (0.65/s)       


In [ ]:
data

,date,price,pettm,price_to_owner_earnings,ps_ratio,pb_ratio,peg_ratio,yield,enterprise_value_to_revenue,enterprise_value_to_ebitda,enterprise_value_to_ebit,earning_yield_greenblatt,rate_of_return_value,shiller_pe_ratio,price_to_tangible_book,price_to_free_cash_flow,price_to_operating_cash_flow
0,1982-03-02,0.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1982-03-03,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1982-03-04,0.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1982-03-05,0.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1982-03-08,0.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10387,2023-05-15,502.04,36.89,45.19,0.95,9.77,NaN,0.72,0.93,21.39,26.44,3.78,NaN,54.75,9.77,40.76,23.41
10388,2023-05-16,496.15,36.45,44.66,0.94,9.65,NaN,0.73,0.92,21.13,26.13,3.83,NaN,54.11,9.65,40.28,23.13
10389,2023-05-17,495.00,36.37,44.55,0.94,9.63,NaN,0.73,0.92,21.08,26.07,3.84,NaN,53.98,9.63,40.19,23.08
10390,2023-05-18,498.10,36.60,44.83,0.94,9.69,NaN,0.72,0.92,21.22,26.23,3.81,NaN,54.32,9.69,40.44,23.22


In [ ]:
data = pd.read_csv(symbol+".csv")

In [ ]:
symbol=""
data.replace(0, np.nan, inplace=True)
metrics_correlation = data.corr()['price'].iloc[1:]
metrics_correlation_abs = metrics_correlation.map(abs)
metrics_correlation_abs = pd.DataFrame(metrics_correlation_abs)
metrics_correlation = pd.DataFrame(metrics_correlation)
metrics = pd.concat([metrics_correlation,metrics_correlation_abs],axis=1)
metrics.columns=['Correlation', 'abs']
metrics = metrics.sort_values(by="abs",ascending=False)
metrics.to_csv(symbol+"_results.csv")
print(metrics)

                              Correlation       abs
price                            1.000000  1.000000
price_to_tangible_book           0.938530  0.938530
pb_ratio                         0.938478  0.938478
peg_ratio                        0.906292  0.906292
enterprise_value_to_revenue      0.874622  0.874622
ps_ratio                         0.859515  0.859515
enterprise_value_to_ebitda       0.777684  0.777684
pettm                            0.616404  0.616404
earning_yield_greenblatt        -0.593954  0.593954
enterprise_value_to_ebit         0.592068  0.592068
price_to_operating_cash_flow     0.577808  0.577808
shiller_pe_ratio                 0.562894  0.562894
rate_of_return_value            -0.524376  0.524376
yield                           -0.523558  0.523558
price_to_free_cash_flow         -0.149740  0.149740
price_to_owner_earnings         -0.147212  0.147212


# Script for symbol

In [ ]:
indicators = pd.read_csv("indicators.csv")
indicators
indicator = indicators.iloc[4:20].key
indicators_list = list(indicators['key'])
indicator = indicators_list[0:10]
indicator = indicators_list[0:10]
aux = ["enterprise_value_to_revenue","enterprise_value_to_ebitda","enterprise_value_to_ebit","earning_yield_greenblatt","rate_of_return_value","shiller_pe_ratio","price_to_tangible_book"
       ,"price_to_free_cash_flow","price_to_operating_cash_flow"]
for i in aux:
    indicator.append(i)
indicator.pop(0)
indicator.pop(0)
indicator.pop(0)
a =indicator.pop(0)
#Please run this command if this is the first time getting the information, if not run get results Correlation
def getCorrelations(symbol):
    indicators = pd.read_csv("indicators.csv")
    api_token = "55bb5f7a051cefa4483b892d4115fb49:afd837a03ca494a823f6b20cb016f142"
    response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/price?type=a')
    content = response.content
    price_data = json.loads(content.decode('utf8'))
    if(len(price_data) == 0 or type(price_data) is dict):
        print("Could not get price data")
        return
    price = pd.DataFrame(price_data)
    price.columns = ["date",'price']
    price["date"] = pd.to_datetime(price["date"])
    data=price
    with alive_bar(len(indicator),force_tty=True) as bar:
        for i in range(len(indicator)):
            bar()
            ind = indicator[i]
            response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/{ind}?type=a')
            content = response.content
            indicator_data = json.loads(content.decode('utf8'))
            if(len(indicator_data)==0):
                print(f"Impossible to found data for {symbol} for metric: {ind}")
            else:
                indicator_df = pd.DataFrame(indicator_data)
                indicator_df[0] = pd.to_datetime(indicator_df[0])
                indicator_df.columns = ["date",ind]
                data = pd.merge(data,indicator_df,how="left",on=["date"])
    data.to_csv(symbol+".csv")
    data = pd.read_csv(symbol+".csv")
    data.replace(0, np.nan, inplace=True)
    metrics_correlation = data.corr()['price'].iloc[1:]
    metrics_correlation_abs = metrics_correlation.map(abs)
    metrics_correlation_abs = pd.DataFrame(metrics_correlation_abs)
    metrics_correlation = pd.DataFrame(metrics_correlation)
    metrics = pd.concat([metrics_correlation,metrics_correlation_abs],axis=1)
    metrics.columns=['Correlation', 'abs']
    metrics = metrics.sort_values(by="abs",ascending=False)
    metrics.to_csv(symbol+"_results.csv")
    metrics = pd.read_csv(symbol+"_results.csv")
    print(f"The best historical metric is {metrics.iloc[1][0]} with correlation {metrics.iloc[1]['Correlation']}")

In [ ]:
def getCorrelationResults(symbol):
    metrics = pd.read_csv(symbol+"_results.csv")
    print(f"The best historical metric is {metrics.iloc[1][0]} with correlation {metrics.iloc[1]['Correlation']}")

In [ ]:
!!pwd

['/home/robdres/gurufocus/GuruFocusAPI']

In [ ]:
metrics = pd.read_csv("COST_results.csv")
print(metrics)

  Unnamed: 0  Correlation  abs
0      price          1.0  1.0


In [ ]:
def getResults(symbol):
    print("Getting data for " + symbol) 
    print()
    if(not os.path.exists(symbol+".csv")):
        getCorrelations(symbol)
    else:
        getCorrelationResults(symbol)
    print("------------------------------------------------------------------")

# Getting results

In [ ]:
import os
for i in symbols:
    getResults(i)

Getting data for AAPL

The best historical metric is pb_ratio with correlation 0.9572152281081412
------------------------------------------------------------------
Getting data for ANET

The best historical metric is shiller_pe_ratio with correlation 0.8767241559003951
------------------------------------------------------------------
Getting data for ASML

The best historical metric is price_to_tangible_book with correlation 0.8168078798994568
------------------------------------------------------------------
Getting data for CHGCY

The best historical metric is pb_ratio with correlation 0.9578621580311536
------------------------------------------------------------------
Getting data for KO

The best historical metric is price_to_tangible_book with correlation 0.5270023023002142
------------------------------------------------------------------
Getting data for COST

The best historical metric is price_to_tangible_book with correlation 0.9385298249093126
----------------------------

# Funcionalidad

Para hacer uso correr todas las celdas de esta sección para tener las funciones activas, e ir a la ultima sección

In [ ]:
!pip3 install alive_progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from alive_progress import alive_bar
import numpy as np

api_token = "<api-token>"

In [ ]:
aux = ['pettm', 'price_to_owner_earnings', 'ps_ratio', 'pb_ratio', 'peg_ratio', 'yield', 'enterprise_value_to_revenue', 'enterprise_value_to_ebitda', 'enterprise_value_to_ebit', 'earning_yield_greenblatt', 'rate_of_return_value', 'shiller_pe_ratio', 'price_to_tangible_book', 'price_to_free_cash_flow', 'price_to_operating_cash_flow']
indicator = aux

#Script para hacer un requests 
def getCorrelations(symbol):
    response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/price?type=a')
    content = response.content
    price_data = json.loads(content.decode('utf8'))
    if(len(price_data) == 0 or type(price_data) is dict):
        print("Could not get price data")
        return
    price = pd.DataFrame(price_data)
    price.columns = ["date",'price']
    price["date"] = pd.to_datetime(price["date"])
    data=price
    with alive_bar(len(indicator),force_tty=True) as bar:
        for i in range(len(indicator)):
            bar()
            ind = indicator[i]
            response = requests.get(f'https://api.gurufocus.com/public/user/{api_token}/stock/{symbol}/{ind}?type=a')
            content = response.content
            indicator_data = json.loads(content.decode('utf8'))
            if(len(indicator_data)==0):
                print(f"Impossible to found data for {symbol} for metric: {ind}")
            else:
                indicator_df = pd.DataFrame(indicator_data)
                indicator_df[0] = pd.to_datetime(indicator_df[0])
                indicator_df.columns = ["date",ind]
                data = pd.merge(data,indicator_df,how="left",on=["date"])
    data.to_csv(symbol+".csv")
    data = pd.read_csv(symbol+".csv")
    data.replace(0, np.nan, inplace=True)
    metrics_correlation = data.corr()['price'].iloc[1:]
    metrics_correlation_abs = metrics_correlation.map(abs)
    metrics_correlation_abs = pd.DataFrame(metrics_correlation_abs)
    metrics_correlation = pd.DataFrame(metrics_correlation)
    metrics = pd.concat([metrics_correlation,metrics_correlation_abs],axis=1)
    metrics.columns=['Correlation', 'abs']
    metrics = metrics.sort_values(by="abs",ascending=False)
    metrics.to_csv(symbol+"_results.csv")
    metrics = pd.read_csv(symbol+"_results.csv")
    print(f"The best historical metric is {metrics.iloc[1][0]} with correlation {metrics.iloc[1]['Correlation']}")

#Function if already data requested
def getCorrelationResults(symbol):
    metrics = pd.read_csv(symbol+"_results.csv")
    print(f"The best historical metric is {metrics.iloc[1][0]} with correlation {metrics.iloc[1]['Correlation']}")
    
#Function to run and get the metrics for the best
def getResults(symbol):
    print("Getting data for " + symbol) 
    print()
    if(not os.path.exists(symbol+".csv")):
        getCorrelations(symbol)
    else:
        getCorrelationResults(symbol)
    print("------------------------------------------------------------------")

# Función para conseguir los valores de mlas métricas y mejor correlación
En este caso como google colab no permite tener guardado archivos, cada vez que se llame a esta función se va a descargar los datos e utilizar 20 requests de gurufocus API!.
Por lo que se recomienda que si ya se tienen los datos para ese símbolo no se debería volver a hacer la llamada. Dentro de este repositorio se encuentran los símbolos que ya se corrieron
https://github.com/Robdres/gurufocus/tree/main/GuruFocusAPI
Entonces si se quiere encontrar los resultados para un símbolo se recomienda revisar si ya está dentro del repositorio.

In [ ]:
api_token = "<api-token>"
symbol = "<symbol>"

getResults(symbol)